In [83]:
import SimpleITK as sitk
import numpy as np
from tkinter.filedialog import askopenfilenames
import cv2


filepath1='/home/rtv/workspace/BVR/resource/Data/P2PGS/DCM/I0000004'
filepath2='/home/rtv/workspace/BVR/resource/Data/P2PGS/DCM/I0000005'

imgno=48

reader=sitk.ImageFileReader()

reader.SetFileName(filepath1)
Data1=reader.Execute()

reader.SetFileName(filepath2)
Data2=reader.Execute()

Data1.


# if Data1.HasMetaDataKey('0018|1164') is not True:
#     Data2.SetMetaData('0018|1164',Data1.GetMetaData('0018|1164'))
# if Data2.HasMetaDataKey('0018|1164') is not True:
#     Data1.SetMetaData('0018|1164',Data2.GetMetaData('0018|1164'))

Image (0x37430c0)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 1
  Modified Time: 6605
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 6590
  UpdateMTime: 6604
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 84]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 84]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 84]
  Spacing: [0.278875, 0.278875, 66.6667]
  Origin: [0, 0, 0]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.278875 0 0
0 0.278875 0
0 0 66.6667

  PointToIndexMatrix: 
3.58584 0 0
0 3.58584 0
0 0 0.015

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x3746a00)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Re

In [5]:
DEGTORAD = np.pi / 180.0

def Initial_transform_Matrix(Data):
    if type(Data) is sitk.SimpleITK.Image:
        DicomData=Data
    elif type(Data) is str:
        reader=sitk.ImageFileReader()
        reader.SetFileName(Data)
        DicomData=reader.Execute()

    if DicomData.HasMetaDataKey('0008|0070') is not True:
        print('not DICOM image, '+type(DicomData))
    elif DicomData.GetMetaData('0008|0070')!='Siemens ':
        print('not Siemens, but '+Data1.GetMetaData('0008|0070'))
    else:
        # print('return Transform Matrix')
        SID=float(DicomData.GetMetaData('0018|1110'))
        SOD=float(DicomData.GetMetaData('0018|1111'))
        # mag=float(DicomData.GetMetaData('0018|1114'))
        angle1=float(DicomData.GetMetaData('0018|1510'))*DEGTORAD
        angle2=float(DicomData.GetMetaData('0018|1511'))*DEGTORAD
        skew=0

        ''' if DicomData.HasMetaDataKey('0018|1164') is not True:
            pixel_spacing=([0.278875,  0.278875])
            
        else:
            pixel_spacing=np.array(DicomData.GetMetaData('0018|1164').split('\\')).astype(float) '''
        pixel_spacing=np.array(DicomData.GetMetaData('0018|1164').split('\\')).astype(float)
        roty=np.array([[np.cos(angle1),0.0,np.sin(angle1)],[0.0,1,0.0],[-np.sin(angle1),0.0,-np.cos(angle1)]])
        rotx=np.array([[1.0,0.0,0.0],[0.0,np.cos(angle2),-np.sin(angle2)],[0.0,np.sin(angle2),np.cos(angle2)]])

        rot=np.matmul(rotx,roty)

        rot=np.vstack((rot,[0.0,0.0,0.0]))
        # print(rot)

        # K=np.array([[mag/(pixel_spacing[0]),mag/(pixel_spacing[1])*skew,0],[0,mag/(pixel_spacing[1]),0],[0,0,1]])
        K=np.array([[SID/(pixel_spacing[0]),SID/(pixel_spacing[1])*skew,256,SOD*256],[0,SID/(pixel_spacing[1]),256,SOD*256],[0,0,1,SOD]])
        # tls=np.zeros([3,1])
        tls=np.array([[0.0],[0.0],[0.0],[1.0]])
        print('angle is '+str(angle1/DEGTORAD)+','+str(angle2/DEGTORAD))

        P=np.matmul(K,np.concatenate((rot,tls),axis=1))
        return P, K, rot, tls

In [6]:
# [P1 , K1, rot1, tls1] =Initial_transform_Matrix(Data1)
[P1 , K1, rot1, tls1] =Initial_transform_Matrix(filepath1)
print(P1,rot1)

# [P2 , K2, rot2, tls2] =Initial_transform_Matrix(Data2)
[P2 , K2, rot2, tls2] =Initial_transform_Matrix(filepath2)
print(P2)

angle is 90.1,-0.1
[[ -2.62651964e+02  -4.46804062e-01   3.81218461e+03   1.91999996e+05]
 [ -2.62651954e+02   3.81129100e+03   4.58414604e-01   1.91999996e+05]
 [ -9.99996954e-01  -1.74532837e-03   1.74532571e-03   7.49999983e+02]] [[ -1.74532837e-03   0.00000000e+00   9.99998477e-01]
 [ -1.74532571e-03   9.99998477e-01   3.04617110e-06]
 [ -9.99996954e-01  -1.74532837e-03   1.74532571e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]
angle is -40.6,22.3
[[  3.16263597e+03   9.71407768e+01  -2.75843060e+03   1.88129236e+05]
 [ -8.24325250e+02   3.76314432e+03   9.61756772e+02   1.88129236e+05]
 [  6.02102630e-01   3.79456160e-01  -7.02485192e-01   7.34879827e+02]]


In [72]:
def global_line_from_image_point(point,Data):
    
    [P, K, rot, tls]=Initial_transform_Matrix(Data)
    [u,v]=point
    coef1=np.array([[u/K[0][0], v/K[1][1], 1.0]])
    coef2=K[2][3]*np.array([[u/K[0][0], v/K[1][1], 0.0]])
#     print(coef2)
    coef1_global=np.matmul(np.linalg.inv(rot[0:3][:]),coef1.T)
    coef2_global=np.matmul(np.linalg.inv(rot[0:3][:]),(coef2.T-tls[0:3]))
#     print((coef2.T-tls[0:3]))
    
    return coef1_global, coef2_global
    

In [77]:
def projected_epipolar_line(point,Data_from,Data_to):
    [coef1_global, coef2_global]=global_line_from_image_point(point,Data_from)
    [P, K, rot, tls]=Initial_transform_Matrix(Data_to)
    coef1=np.matmul(rot[0:3][:],coef1_global)
    coef2=np.matmul(rot[0:3][:],coef2_global)+tls[0:3]
    coef1=np.vstack([coef1,1])
    coef2=np.vstack([coef2,1])
    print(coef1_global)
    pixel1=np.matmul(P,coef2)
    pixel1/=pixel1[2]
    pixel2=np.matmul(P,coef1+coef2)
    pixel2/=pixel2[2]
    coef_img=np.array([pixel2[1]-pixel1[1], pixel1[0]-pixel2[0], pixel2[0]*pixel1[1]-pixel1[0]*pixel2[1]])
    
    return coef_img
    

In [7]:
print(np.matmul(P1,[100,0,0,1])/np.matmul(P1[2][:],[100,0,0,1]))
print(np.matmul(P1,[0,100,0,1])/np.matmul(P1[2][:],[0,100,0,1]))
print(np.matmul(P1,[0,0,100,1])/np.matmul(P1[2][:],[0,0,100,1]))
print(np.matmul(P2,[100,0,0,1])/np.matmul(P2[2][:],[100,0,0,1]))
print(np.matmul(P2,[0,100,0,1])/np.matmul(P2[2][:],[0,100,0,1]))
print(np.matmul(P2,[0,0,100,1])/np.matmul(P2[2][:],[0,0,100,1]))

[ 254.97650134  254.9765029     1.        ]
[ 256.          764.35001716    1.        ]
[ 764.11347566  256.0015478     1.        ]
[ 634.38450424  132.93677281    1.        ]
[ 256.          730.36372341    1.        ]
[-131.97365684  427.76334732    1.        ]


In [78]:
point=[100,200]
[u,v]=point
# print(rot1)
# [P, K, rot, tls]=Initial_transform_Matrix(filepath1)
# [coef1, coef2]=global_line_from_image_point(point,filepath1)
# pixel1=np.matmul(P,coef2)
# pixel1/=pixel1[2]
# pixel2=np.matmul(P,coef1+coef2)
# pixel2/=pixel2[2]
# coef_img=np.array([[pixel2[1]-pixel1[1], pixel1[0]-pixel2[0], pixel2[0]*pixel1[1]-pixel1[0]*pixel2[1]]])
coef_img=projected_epipolar_line(point,filepath1,filepath2)
    
print(coef_img)

angle is 90.1,-0.1
angle is -40.6,22.3
[[-1.00004883]
 [ 0.05072402]
 [ 0.02448934]]
[[ -1.11846611e+02]
 [ -2.72201001e+01]
 [  3.55252059e+04]]


In [64]:
print(np.vstack([coef2,1]))

[[ -3.43412455e-02]
 [  3.93520088e+01]
 [  1.96760044e+01]
 [  1.00000000e+00]]
